# How to get thermophysical properties from NIST online database

NIST provides a large data set of fluid properties that are derived from various sources.
The provided properties include 
*   Density
*   Heat Capacity (C$_P$ and C$_V$)
*   Viscosity
*   Thermal conductivity

and several others. For copyright reasons, the downloaded data cannot be shipped with the pandapipes package directly.
A common, manual way to get the data is through the webhook "Thermophysical Properties of Fluid
Systems": https://webbook.nist.gov/chemistry/fluid/. An automated approach to download and
process several isothermal curves (the underlying tables) is introduced in this tutorial.
For further information on the underlying state equations and scientific references, please refer
 to the web interface.


## High level convenience function
First, we import the main function called `create_database_from_nist`

In [3]:
from pandapipes.properties.get_nist_properties_data import create_database_from_nist

Then, a fluid has to be chosen. Only "long" names are supported, no chemical formulas. Currently,
 hydrogen and methane are implemented.

In [ ]:
chemical = "hydrogen"

 Later in this tutorial you will see how to add other fluids.

Other parameters determine the queried pressure and temperature range as
well as the step width and units of the properties. Except for `chemical`, all parameters have
reasonable default values and do not need to be set explicitly. For clarity, we do so anyway:

In [ ]:
t_min_k = 263.15
t_max_k = 333.15
t_step_k = 10

p_min = 1
p_max = 200
p_step = 4
p_unit = "bar"

If you like, you can set a specific target folder for the downloaded csv-files by specifing
`target_folder`.
The default target folder is pandapipes/properties/\[chemical name\].

In [ ]:
target_folder = "prop_example_folder"  # in the tutorials folder

By default, the isothermal curves are finally summarized in a pandas-dataframe. This dataframe is
 the return value of the function. If you want to store it too (recommended), set `save_summary =
  True`

To get the data, simply run

In [ ]:
df = create_database_from_nist(chemical, t_min_k=t_min_k, t_max_k=t_max_k, t_step_k=t_step_k,
                              p_min=p_min, p_max=p_max, p_step=p_step, p_unit=p_unit,
                              target_folder=target_folder, save_summary=True)

The returned dataframe covers all available properties for the fluid for the set pressure and
temperature range. Internally, the parameters are converted to an URL-string for each temperature
 level. These URLs are called and the answers are converted to csv-files which are stored in
 `target_folder`. The dataframe looks like this:

In [ ]:
df

## Column names
Please note that by default the column names of the resulting dataframe differ from the usual
pandapipes notation. You might have to do some renaming, e.g. like this:

In [ ]:
pandapipes_names = {'Density (kg/m3)': 'density',
                    'Cp (J/mol*K)': 'heat_capacity',
                    'Viscosity (uPa*s)': 'viscosity'}
df.rename(pandapipes_names, axis='columns', inplace=True)

You can use this dataframe to create a two dimensional (pressure and temperature) fluid property.
Or you can simply create the standard text files that are originally used for
properties in pandapipes with the following function. The function
`create_textfiles_from_nist_summary` looks for nist_summary.csv in `folder` and
 extracts density, viscosity and heat capacity at a given pressure for all available temperatures.
 Usually, the properties in the text files are at 1 bar. Please note that the values for
  compressibility (which are required for uni-dimensional pipeflow calculation) are currently not
   automatically calculated. In this case, the compressibility values have to be added manually.


In [ ]:
from pandapipes.properties.get_nist_properties_data import create_textfiles_from_nist_summary
create_textfiles_from_nist_summary(folder=target_folder, at_pressure_bar=1)

## Add a new chemical
If you want to download data that is not yet implemented but available in the fluid
drop down selection on https://webbook.nist.gov/chemistry/fluid/ , you can add extend the code.

First, look up the CAS-registry number (a unique number code) for the chemical, e.g. on wikipedia
 or here https://webbook.nist.gov/chemistry/name-ser/ . Then, add this number without dashes to the
 dictionary `cas_number_dict` in `download_as_csv_from_nist` that maps names and CAS-numbers.
 Finally, you can run `create_database_from_nist` as written before with the new `chemical` name.
  There is no query for molar_mass yet. However, https://webbook.nist.gov/chemistry/name-ser/
  gives you the molecular weight, too, so molar_mass.txt can be added manually.

## Create a summarizing dataframe from existing csv-files
If you want to summarize csv files that have already been downloaded to `folder`, e.g. from
earlier queries, you can use the following function:

In [ ]:
from pandapipes.properties.get_nist_properties_data import merge_nist_csvs_in_folder
folder = "prop_example_folder"
df = merge_nist_csvs_in_folder(folder)